In [ ]:
'''
Okay the plan;



1. We predict with sahi on the whole ortho image using different parameters (overlap, slice sizing) 
3. export each prediction as a seperate folder / yolo file format
2. We tile each prediction result image and bounding boxes into smaller images
    - this would basically be a dataset of original sized images along with a corresponding labels folder and a YAML file
3. randomly select 10% of images and label them in labelstudio
4. use fiftyone merge datasets to merge predict output tiles and labels with labelstudio groundtruth label and tiles
5. evaluate model performace and adjust confidence intervals
6. georef and make figures using the whole image predictions




'''

In [1]:
# first check the wd
# this should be AIggregation folder

import os
os.getcwd()

#os.chdir("..")


'c:\\Users\\obiew\\Desktop\\github\\AIggregation'

In [4]:

# import fiftyone
import fiftyone as fo
from fiftyone import ViewField as F


# Import ortho image into fiftyone dataset type
dataset = fo.Dataset.from_images(
    ["datasets/drone_ortho/ortho_clip_23april.png"]
)


# open test image instance to verify it loaded correctly
session = fo.launch_app(dataset)

 100% |█████████████████████| 1/1 [22.2ms elapsed, 0s remaining, 50.7 samples/s] 


In [5]:
# import sahi 
from sahi import AutoDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction


# specify AI detection model
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov5',
    model_path='AIggregation_yolov5m/weights/best.pt', #specify path to trained model
    confidence_threshold=0.25,
    device="cpu", # or 'cuda:0'
)


YOLOv5  2024-7-11 Python-3.12.3 torch-2.3.1+cpu CPU

c:\Users\obiew\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
WARNING  DetectMultiBackend failed: cannot instantiate 'PosixPath' on your system


TypeError: ('model_path is not a valid yolov5 model path: ', NotImplementedError("cannot instantiate 'PosixPath' on your system"))

In [5]:


# Run and compare different slicing parameters
def predict_with_slicing(sample, label_field, **kwargs):
    result = get_sliced_prediction(
        sample.filepath, detection_model, verbose=0, **kwargs
    )
    sample[label_field] = fo.Detections(detections=result.to_fiftyone_detections())


#kwargs if you want to specify many arguments that are the same for all models
#kwargs = {"overlap_height_ratio": 0.2, "overlap_width_ratio": 0.2}

kwargs = {}


for sample in dataset.iter_samples(progress=True, autosave=True):
    predict_with_slicing(sample,
                         label_field="small_slices",
                         slice_height=304,
                         slice_width=304,overlap_height_ratio = .2,
                         overlap_width_ratio=.2,
                         **kwargs
    )
    predict_with_slicing(sample,
                         label_field="large_slices",
                         slice_height=608, slice_width=608,
                         overlap_height_ratio = .2, overlap_width_ratio=.2,
                         **kwargs
    )
    predict_with_slicing(sample,
                         label_field="large_slices_large_overlap",
                         slice_height=608, slice_width=608,
                         overlap_height_ratio = .4, overlap_width_ratio=.4,
                         **kwargs
    )


#bring up new session to see boxes
session.show()



 100% |█████████████████████| 1/1 [25.7m elapsed, 0s remaining, 0.0 samples/s] 


07/18/2024 08:30:35 - INFO - eta.core.utils -    100% |█████████████████████| 1/1 [25.7m elapsed, 0s remaining, 0.0 samples/s] 


In [ ]:
# tile the image and outputs


''' 
the image can then easily be tiled. The bouding boxes would be currently saved as layers each with a title (eg. large_slices)

We can extract each layer of bounding boxes in yolo formate 
(basically csv listing top left corner of box relative to pixel x/y max and then size of bounding box x/y)

we could then calculate any values where location in image+dimension of bounding box fall within our tile pixel values

We would then write a script to save the tiled image and corresponding bounding box coordinates (in proportion of x/y max) into a folders
then create a YAML in that folder

then reimport, merge on a per image bases, and evaluate between the different predictions
'''


# for tiling
# tile images and save list of tile coordinates
# convert yolo coord to pixel coordinates
# Extract pixel coords that fall within specific tiles 
# loop through saving as own json file named same as image



# save different outputs

# create a YAML file

# import and merge them back together

In [ ]:

# evaluate 
'''


# evaluate the different sahi predictions
large_slice_results = dataset.evaluate_detections("large_slices", gt_field="ground_truth", eval_key="eval_large_slices")
small_slice_results = dataset.evaluate_detections("small_slices", gt_field="ground_truth", eval_key="eval_small_slices")
overlap_slice_results = dataset.evaluate_detections("large_slices_overlap", gt_field="ground_truth", eval_key="eval_large_slices_overlap")

# print eval results
print("-" * 50)
print("Large slice results:")
large_slice_results.print_report()

print("-" * 50)
print("Large slice overlap results:")
overlap_slice_results.print_report()

print("-" * 50)
print("Small slice results:")
small_slice_results.print_report()

'''